<a href="https://colab.research.google.com/github/pvanand07/GPT-Research-tool/blob/master/gpt_research_tool_main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U google-generativeai -q
# Utilizes quantized models for fast inference on cpu
!pip install qdrant-client[fastembed] -q

In [2]:
research_questions = [
    "Identify the industry in which Canoo ev operates, along with its size, growth rate, trends, and key players.",
    "Analyze Canoo ev's main competitors, including their market share, products or services offered, pricing strategies, and marketing efforts.",
    "Identify key trends in the ev market, including changes in consumer behavior, technological advancements, and shifts in the competitive landscape.",
    "Gather information on Canoo ev's financial performance, including its revenue, profit margins, return on investment, and expense structure."
]

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import userdata
import requests
from IPython.display import display
from IPython.display import Markdown
import pathlib
import textwrap
import google.generativeai as genai
from googleapiclient.discovery import build
from nltk.tokenize import sent_tokenize
import ast
import json
from qdrant_client import QdrantClient


my_api_key = userdata.get('google_search_api') # Get your Google Search API key from here (free) (LIMIT:100 queries/day): https://developers.google.com/custom-search/v1/overview
my_cse_id = userdata.get('cse_id') # Obtain your Custom Search Engine ID from here (free): https://developers.google.com/custom-search/v1/overview
GEMINI_API_KEY = userdata.get('gemini_api') # Get your GEMINI API key from here (free) (LIMIT:100 queries/hour) https://aistudio.google.com/app/apikey

In [5]:
def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def ask_gemini(question, GEMINI_API_KEY):

  genai.configure(api_key=GEMINI_API_KEY)
  model = genai.GenerativeModel('gemini-pro')
  response = model.generate_content(question)
  display(to_markdown(response.text))

  return response.text

In [7]:
def generate_search_queries_prompt(question, max_iterations=6):
    """ Generates the search queries prompt for the given question.
    Args: question (str): The question to generate the search queries prompt for
    Returns: str: The search queries prompt for the given question
    """

    return 'Follow the instructions'\
           f'1. Write {max_iterations} google search queries to search online that form an objective opinion from the following: "{question}"' \
           f'2. output only the list of search queries in the format: ["query 1", "query 2", ...].'


def generate_report_prompt(question, context, report_format="apa", total_words=1000):
    """ Generates the report prompt for the given question and research summary.
    Args: question (str): The question to generate the report prompt for
            research_summary (str): The research summary to generate the report prompt for
    Returns: str: The report prompt for the given question and research summary
    """

    return f'Using the Information: """{context}"""\n\n' \
           f'answer the following' \
           f'query : "{question}" in a detailed report --' \
           f"with facts and numbers if available and a minimum of {total_words} words.\n" \
           f"You MUST write the report in {report_format} format.\n "

In [174]:
def generate_search_queries(research_questions):
  research_queries_dict ={}
  for question in research_questions:
    question_prompt = generate_search_queries_prompt(question)
    research_queries_dict[question] = ask_gemini(question_prompt,GEMINI_API_KEY)

  return research_queries_dict


def generate_research_query_list(research_queries_dict):
  # Convert Gemini output from string to list
  research_queries_dict_list = {}
  for question, research_queries in research_queries_dict.items():
    if research_queries and research_queries.strip():  # Check if the input is not null or empty
        try:
            research_queries_dict_list[question] = ast.literal_eval(research_queries)

        except SyntaxError or ValueError:
            # If ast.literal_eval() fails due to syntax error, tokenize the string into sentences using NLTK
            research_queries_dict_list[question] = sent_tokenize(research_queries)
            if len(research_queries_dict_list[question]) == 1:
              research_queries_dict_list[question] = research_queries.split('\n')
    else:
        print("Input ask_gemini(question)-->research_queries is null or empty. Passing...")

  return research_queries_dict_list

research_queries_dict = generate_search_queries(research_questions)
research_queries_dict_list = generate_research_query_list(research_queries_dict)

> ["Industry in which Canoo ev operates", "Size of the electric vehicle industry", "Electric vehicle industry growth rate", "Trends in the electric vehicle industry", "Key players in the electric vehicle industry", "Market research on Canoo ev"]

> ["Canoo EV competitors market share", "Canoo EV competitors products and services", "Canoo EV competitors pricing strategies", "Canoo EV competitors marketing efforts", "Comparative analysis of Canoo EV competitors", "Canoo EV competition landscape"]

> ["Consumer behavior in the electric vehicle market", "Technological advances in electric vehicles", "Competitive landscape of the electric vehicle market", "Trends in electric vehicle ownership", "Shifting market dynamics for electric vehicles", "Emerging technologies in the electric vehicle sector"]

> ["Canoo Revenue", "Canoo Profit Margin", "Canoo Return on Investment", "Canoo Expense Structure", "Canoo Financial Performance Analysis", "Canoo Earnings Reports"]

In [ ]:
def extract_data_from_google_search_results(research_queries_dict_list):
# Using the search queries to perform google search and extract links
  research_links_google = {}

  for question, research_queries_list in research_queries_dict_list.items():
    for research_query in research_queries_list:
      research_links_google[research_query] = google_search(research_query, my_api_key, my_cse_id, num=2)

  # Writing google search results dictionary to a JSON file
  with open("research_links_google.json", 'w') as json_file:
      json.dump(research_links_google, json_file)

  print(f"Dictionary saved as JSON file: research_links_google.json")

  return research_links_google

research_links_google = extract_data_from_google_search_results(research_queries_dict_list)

In [184]:
def extract_links(research_links_google):
  # Load the search results from the JSON file
  # File path to the JSON file
  file_path = "/content/research_links_google.json"

  # Loading JSON file as dictionary
  with open(file_path, 'r') as json_file:
      research_links_google1 = json.load(json_file)


  topics_dict_list = [research_links_google[k] for k, v in research_links_google.items()] # 24 Topics x 2 URLS

  links_list = []
  for topic_dict in topics_dict_list:
    try:
      for item in topic_dict['items']:
        links_list.append(item['link'])
    except:
      pass

  return links_list

links_list = extract_links(research_links_google)

In [186]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import requests
from bs4 import BeautifulSoup

class Scraper:
    """
    Scraper class to extract the content from web page links.
    """
    def __init__(self, urls, user_agent):
        """
        Initialize the Scraper class.
        Args:
            urls (list): List of URLs to scrape.
            user_agent (str): User agent string for the HTTP header to simulate a browser request.
        """
        self.urls = urls
        self.session = requests.Session()
        self.session.headers.update({
            "User-Agent": user_agent
        })

    def run(self):
        """
        Extracts the content from the provided URLs.
        Returns:
            list: A list of dictionaries with URL and its corresponding raw content.
        """
        partial_extract = partial(self.extract_data_from_link, session=self.session)
        with ThreadPoolExecutor(max_workers=20) as executor:
            contents = executor.map(partial_extract, self.urls)
        res = [content for content in contents if content['raw_content']]
        return res

    def extract_data_from_link(self, link, session):
        """
        Extracts the data from a single web page link.
        Args:
            link (str): URL to scrape.
            session (requests.Session): Session object for making requests.
        Returns:
            dict: A dictionary with URL and its raw content.
        """
        try:
            response = session.get(link, timeout=4)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'lxml', from_encoding=response.encoding)
                raw_content = self.scrape_text_with_bs(soup)
                if len(raw_content) < 100:
                    return {'url': link, 'raw_content': None}
                return {'url': link, 'raw_content': raw_content}
            return {'url': link, 'raw_content': None}
        except Exception as e:
            return {'url': link, 'raw_content': None}

    def scrape_text_with_bs(self, soup):
        """
        Extracts and cleans text content from a BeautifulSoup object.
        Args:
            soup (BeautifulSoup): BeautifulSoup object of a web page.
        Returns:
            str: Cleaned text content from the web page.
        """
        for script_or_style in soup(["script", "style"]):
            script_or_style.extract()  # Remove script and style elements

        text = soup.get_text(separator="\n")
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        content = "\n".join(chunk for chunk in chunks if chunk)
        return content


In [206]:
def extract_and_store_data(links_list, user_agent, output_file):
    """
    Extracts data from the provided URLs and stores the data as a JSON file.

    Args:
        research_links_google_list (dict): A dictionary with keys as topics and values as lists of URLs.
        user_agent (str): User agent string for HTTP requests.
        output_file (str): Path to the output JSON file.
    """

    # Initialize the Scraper with URLs and a user agent
    scraper = Scraper(links_list, user_agent)
    # Extract data
    data = scraper.run()

    data_dict = {}

    for item in data:
      data_dict[item['url']] = item['raw_content']
    data_dict

    # Write the data to a JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data_dict, f, ensure_ascii=False, indent=4)

# Usage
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
output_file = 'extracted_data.json'  # Specify the path to the output file

# Call the function with the provided research links, user agent, and output file path
extract_and_store_data(links_list, user_agent, output_file)


In [8]:
import json

# File path to the JSON file
file_path = "/content/extracted_data.json"

# Loading JSON file as dictionary
with open(file_path, 'r') as json_file:
    extracted_data = json.load(json_file)
len(extracted_data)

15

In [9]:
%%time
def add_to_qdrant(extracted_data):

  # Initialize the Qdrant client with in-memory storage for demonstration purposes
  client = QdrantClient(":memory:")  # Adjust the host and port if necessary
  client.set_model("sentence-transformers/all-MiniLM-L6-v2")

  # Assuming `extracted_data` is a pre-defined dictionary with your data
  documents = []
  metadatas = []
  ids = []

  # Prepare your documents and metadata
  for url, data in extracted_data.items():
    sentences = data.split('\n')

    # Here you might need to adjust how you handle IDs to ensure they are unique integers
    # Qdrant currently requires numeric IDs
    for i, sentence in enumerate(sentences):
        documents.append(sentence)
        metadatas.append({"url": url+str(i)})
        ids.append(hash(f"{url}_{i}"))  # Example way to generate a numeric ID

  # Add documents to Qdrant
  client.add(
      collection_name="your_collection_name",
      documents=documents,
      metadata=metadatas,
      ids=ids
  )
  return client

client = add_to_qdrant(extracted_data)

100%|██████████| 83.2M/83.2M [00:03<00:00, 22.6MiB/s]


CPU times: user 2min 47s, sys: 1.74 s, total: 2min 49s
Wall time: 2min 53s


In [10]:
def query_index(client,collection_name, query_text, limit=100):
  """ Query the index with the given client and query text.
  """

  search_result = client.query(
  collection_name=collection_name,
  query_text=query_text, limit= limit
  )
  context = "".join(set(["".join(result.metadata['document']) for result in search_result]))

  return context

In [11]:
%%time
def create_report(client, research_questions):
  response = []
  display(to_markdown("# Your Research Report"))
  for question in research_questions:
    context = query_index(client, "your_collection_name", question)
    prompt = generate_report_prompt(question, context[:31000*4])
    response.append(ask_gemini(prompt, GEMINI_API_KEY))
    #response.append(ask_gemini("Write an detailed article about the question: {{" + question + "}} given the context from scraped: {{" + context+ "}}",GEMINI_API_KEY))
    #to_markdown(response)
  return response

response = create_report(client, research_questions)

> # Your Research Report

> **Industry Overview**
> 
> **Industry:** Electric Vehicle (EV) Market
> 
> **Size:**
> 
> * The global EV market is estimated to be valued at $580.2 billion in 2023.
> * The market is projected to grow at a compound annual growth rate (CAGR) of 22.6% from 2023 to 2030, reaching $3,198.2 billion by 2030.
> 
> **Growth Rate:**
> 
> * The EV market is experiencing significant growth due to increasing government initiatives, technological advancements, and rising consumer demand for sustainable transportation solutions.
> 
> **Trends:**
> 
> * **Growing consumer acceptance for EVs:** Consumers are becoming increasingly aware of the environmental benefits and cost savings associated with EVs.
> * **Government support:** Governments worldwide are implementing policies and incentives to promote EV adoption, such as tax credits, purchase rebates, and charging infrastructure investments.
> * **Technological advancements:** EV manufacturers are不斷創新ns, improving battery technology, increasing vehicle range, and reducing charging times.
> * **Expansion of charging infrastructure:** The number of public EV charging stations is growing rapidly globally, making it more convenient for EV owners to charge their vehicles.
> 
> **Key Players:**
> 
> The global EV market is highly competitive, with numerous established automakers and emerging EV startups vying for market share. Some of the key players include:
> 
> * BYD (China)
> * Tesla (US)
> * Volkswagen AG (Germany)
> * SAIC Motors (China)
> * Stellantis (Netherlands)
> * Hyundai Motor Group (South Korea)
> * Ford Motor Company (US)
> * General Motors (US)
> * Toyota Motor Corporation (Japan)
> * Nissan Motor Corporation (Japan)
> * BMW Group (Germany)
> * Daimler AG (Germany)
> * Rivian Automotive (US)
> * Lucid Motors (US)
> * NIO (China)
> * Xpeng Motors (China)
> * Li Auto (China)
> 
> **Canoo's Position in the EV Industry**
> 
> Canoo Inc. is a relatively new entrant to the EV market. The company was founded in 2017 and is headquartered in Bentonville, Arkansas. Canoo designs, develops, and manufactures electric vehicles, including delivery vans, multipurpose delivery vehicles, and lifestyle vehicles.
> 
> Canoo's unique selling proposition is its modular platform architecture, which allows it to quickly and cost-effectively develop and produce a wide range of EV models. The company also offers a subscription-based business model, which provides customers with access to vehicles without the need for long-term ownership.
> 
> Canoo is still in its early stages of growth, but the company has made significant progress in developing its product lineup and establishing partnerships with major players in the EV ecosystem. In August 2023, Canoo announced a strategic partnership with Walmart to purchase 4,500 electric delivery vans, with an option to purchase up to 10,000 in the future.
> 
> **Conclusion**
> 
> The EV industry is experiencing rapid growth and is poised for continued expansion in the coming years. Canoo Inc. is a well-positioned player in this growing market, with its innovative platform architecture, subscription-based business model, and strategic partnerships. As the EV industry matures, Canoo is expected to play an increasingly significant role in shaping the future of transportation.

> **Canoo EV's Main Competitors**
> 
> Canoo is an electric vehicle (EV) manufacturer that is headquartered in Los Angeles, California. The company was founded in 2017 by Ulrich Kranz, Stefan Krause, and Richard Kim. Canoo produces a range of EVs, including the Lifestyle Vehicle, the Multi-Purpose Delivery Vehicle (MPDV), and the Pickup Truck.
> 
> The EV market is highly competitive, with a number of well-established players. Canoo's main competitors include:
> 
> * **Tesla** is the world's leading EV manufacturer. The company was founded in 2003 by Elon Musk and JB Straubel. Tesla produces a range of EVs, including the Model S, Model X, Model 3, and Model Y. Tesla also produces solar panels and energy storage systems.
> * **Rivian** is an American EV manufacturer that was founded in 2009. Rivian produces a range of EVs, including the R1T pickup truck and the R1S SUV. Rivian also produces electric commercial vehicles.
> * **Lucid Motors** is an American EV manufacturer that was founded in 2007. Lucid Motors produces a range of EVs, including the Lucid Air sedan and the Lucid Gravity SUV. Lucid Motors also produces electric powertrains and batteries.
> * **NIO** is a Chinese EV manufacturer that was founded in 2014. NIO produces a range of EVs, including the ES8 SUV, the ES6 SUV, and the EC6 SUV. NIO also produces electric bicycles and electric charging stations.
> * **Xpeng** is a Chinese EV manufacturer that was founded in 2014. Xpeng produces a range of EVs, including the G3 SUV, the P7 sedan, and the P5 sedan. Xpeng also produces electric charging stations.
> * **Li Auto** is a Chinese EV manufacturer that was founded in 2015. Li Auto produces a range of EVs, including the Li ONE SUV and the Li L9 SUV. Li Auto also produces electric charging stations.
> 
> **Market Share**
> 
> Tesla is the market leader in the EV market, with a market share of 20%. Rivian, Lucid Motors, NIO, Xpeng, and Li Auto are all relatively new entrants to the EV market, but they are all growing rapidly. In 2022, Rivian delivered 25,051 vehicles, Lucid Motors delivered 6,610 vehicles, NIO delivered 258,038 vehicles, Xpeng delivered 222,932 vehicles, and Li Auto delivered 145,700 vehicles.
> 
> **Products or Services Offered**
> 
> Canoo's main competitors all offer a range of EVs. Tesla offers the Model S, Model X, Model 3, and Model Y. Rivian offers the R1T pickup truck and the R1S SUV. Lucid Motors offers the Lucid Air sedan and the Lucid Gravity SUV. NIO offers the ES8 SUV, the ES6 SUV, and the EC6 SUV. Xpeng offers the G3 SUV, the P7 sedan, and the P5 sedan. Li Auto offers the Li ONE SUV and the Li L9 SUV.
> 
> In addition to EVs, Tesla also offers solar panels and energy storage systems. Rivian also produces electric commercial vehicles. Lucid Motors also produces electric powertrains and batteries. NIO also produces electric bicycles and electric charging stations. Xpeng also produces electric charging stations. Li Auto also produces electric charging stations.
> 
> **Pricing Strategies**
> 
> Canoo's main competitors all have different pricing strategies. Tesla's EVs are priced from $46,990. Rivian's EVs are priced from $73,000. Lucid Motors' EVs are priced from $87,400. NIO's EVs are priced from $56,571. Xpeng's EVs are priced from $45,990. Li Auto's EVs are priced from $58,990.
> 
> **Marketing Efforts**
> 
> Canoo's main competitors all have different marketing strategies. Tesla is known for its direct-to-consumer sales model and its focus on technology. Rivian is known for its adventure-oriented vehicles and its focus on sustainability. Lucid Motors is known for its luxury EVs and its focus on performance. NIO is known for its community-oriented approach and its focus on innovation. Xpeng is known for its technology-focused vehicles and its focus on affordability. Li Auto is known for its family-oriented vehicles and its focus on safety.
> 
> **Conclusion**
> 
> Canoo is a new entrant to the EV market, but the company has a number of advantages over its competitors. Canoo has a strong team of experienced executives, and the company has developed a number of innovative EVs. Canoo also has a unique business model that is designed to appeal to consumers. If Canoo can execute on its plans, the company has the potential to become a major player in the EV market.

> **Key Trends in the Electric Vehicle (EV) Market**
> 
> **Introduction**
> 
> The electric vehicle (EV) market is experiencing rapid growth due to increasing environmental concerns, government incentives, and technological advancements. This report identifies key trends in the EV market, including changes in consumer behavior, technological advancements, and shifts in the competitive landscape.
> 
> **Changes in Consumer Behavior**
> 
> * **Increasing consumer awareness of environmental issues:** Consumers are becoming increasingly aware of the environmental impact of fossil fuel-powered vehicles and are looking for more sustainable alternatives.
> * **Growing demand for EVs among younger generations:** Younger generations are more likely to be environmentally conscious and are more likely to purchase EVs.
> * **Shift towards ride-sharing and car-sharing:** Ride-sharing and car-sharing services are becoming increasingly popular, which is reducing the need for car ownership and making EVs more accessible to consumers.
> 
> **Technological Advancements**
> 
> * **Improvements in battery technology:** Battery technology is improving rapidly, which is increasing the range and performance of EVs.
> * **Development of new EV models:** Automakers are developing a wide range of new EV models, including sedans, SUVs, and pickup trucks.
> * **Advancements in charging infrastructure:** The charging infrastructure for EVs is expanding rapidly, making it easier for consumers to charge their vehicles.
> 
> **Shifts in the Competitive Landscape**
> 
> * **Entry of new players:** New players are entering the EV market, including startups and traditional automakers.
> * **Collaboration between automakers and technology companies:** Automakers are partnering with technology companies to develop and manufacture EVs.
> * **Government support for EV adoption:** Governments around the world are providing incentives for EV adoption, such as tax breaks and subsidies.
> 
> **Facts and Numbers**
> 
> * The global EV market is expected to grow from $384.65 billion in 2022 to $1,579.10 billion in 2030, at a CAGR of 17.8%.
> * China is the largest market for EVs, accounting for over 50% of global sales.
> * The United States is the second largest market for EVs, with sales growing rapidly.
> * Europe is also a major market for EVs, with several countries offering incentives for EV adoption.
> 
> **Conclusion**
> 
> The EV market is undergoing rapid growth due to a combination of changing consumer behavior, technological advancements, and shifts in the competitive landscape. Consumers are becoming more aware of environmental issues and are looking for more sustainable alternatives to fossil fuel-powered vehicles. Automakers are developing a wide range of new EV models and improving battery technology. Governments around the world are also providing incentives for EV adoption. As a result, the EV market is expected to continue to grow rapidly in the coming years.
> 
> **References**
> 
> * Fortune Business Insights. (2023). Electric Vehicle Market Size, Share & Industry Analysis, By Component (Battery, Motor, Power Electronics, Others), By Vehicle Type (Passenger Cars, Commercial Vehicles), By Vehicle Class (A-Class, B-Class, C-Class, D-Class, Others), By Top Speed (<100 Kmph, 100-150 Kmph, >150 Kmph), By Vehicle Drive Type (BEV, PHEV, FCEV), By EV Charging Point Type (AC Charging, DC Charging, Others), By Vehicle Connectivity (Yes, No), By End Use (Personal Use, Commercial Use), By Propulsion (Battery Electric, Plug-in Hybrid, Fuel Cell), and Region (North America, Europe, Asia Pacific, Middle East & Africa, South America), Forecast 2023-2030.
> * International Energy Agency. (2023). Global EV Outlook 2023.
> * McKinsey & Company. (2022). Electric Vehicles: The Future of Mobility.

> ## Canoo's Financial Performance: An In-Depth Analysis
> 
> ### Introduction
> 
> Canoo Inc. is a Southern California-based electric vehicle manufacturer founded in 2017. The company designs, develops, and manufactures electric vehicles for commercial and personal use. Canoo has a unique business model that focuses on subscription-based ownership rather than traditional vehicle sales.
> 
> ### Revenue
> 
> Canoo's revenue has grown significantly in recent years. In 2022, the company reported revenue of $1.2 million, up from $25 million in 2021. This growth was primarily driven by the increased production and delivery of the company's Lifestyle Vehicle.
> 
> Canoo's revenue is expected to continue to grow in the coming years. The company has a strong order book of over 17,000 vehicles. In addition, Canoo is planning to launch several new vehicles in the coming years, including a pickup truck and a delivery van.
> 
> ### Profitability
> 
> Canoo is not yet profitable, but the company is expected to achieve profitability in 2024. The company's gross profit margin has improved significantly in recent quarters, and Canoo is expected to achieve a positive EBITDA margin in 2024.
> 
> Canoo's profitability will be driven by several factors, including increased production volumes, cost reductions, and the launch of new vehicles. The company is also exploring new revenue streams, such as subscription-based services and software sales.
> 
> ### Return on Investment (ROI)
> 
> Canoo's ROI is difficult to calculate, as the company is not yet profitable. However, the company's strong order book and future growth prospects suggest that Canoo is a good investment for long-term investors.
> 
> ### Expense Structure
> 
> Canoo's expense structure is typical of a pre-revenue company. The company's largest expenses are research and development (R&D) and sales and marketing (S&M).
> 
> Canoo's R&D expenses are primarily related to the development of the company's electric vehicles. The company is also investing in new technologies, such as autonomous driving and battery technology.
> 
> Canoo's S&M expenses are primarily related to the marketing and sale of the company's vehicles. The company is also investing in building a nationwide sales and service network.
> 
> ### Financial Outlook
> 
> Canoo's financial outlook is positive. The company has a strong order book and is expected to achieve profitability in 2024. Canoo is also exploring new revenue streams and is investing in new technologies.
> 
> ### Conclusion
> 
> Canoo is a promising electric vehicle company with a strong financial outlook. The company has a strong order book, is expected to achieve profitability in 2024, and is exploring new revenue streams. Canoo is a good investment for long-term investors.
> 
> ### References
> 
> 1. Canoo Inc. (2023, March 8). Form 10-K. Retrieved from https://www.sec.gov/ix?doc=/Archives/edgar/data/1822341/000119312523049543/d1822341d10k.htm
> 2. Canoo Inc. (2023, August 4). Form 10-Q. Retrieved from https://www.sec.gov/Archives/edgar/data/1822341/000119312523165067/d1822341d10q.htm

In [231]:
with open("report.md", 'w') as report:
  report.write("".join(response))

# FINAL REPORT IN MARKDOWN FORMAT

**Industry Overview**

Canoo EV operates within the electric vehicle (EV) industry, a rapidly growing market driven by increasing demand for sustainable transportation solutions. The industry is characterized by advancements in EV technology, rising consumer awareness, and government initiatives promoting EV adoption.

**Market Size**

As of 2023, the global EV market is estimated to be around 12.8 million units in volume. This market is expected to grow exponentially over the next decade, reaching an estimated 100 million units by 2030.

**Growth Rate**

The EV market is experiencing a significant growth rate, projected to expand at a compound annual growth rate (CAGR) of over 20% from 2023 to 2030. This growth is fueled by factors such as rising fuel prices, environmental concerns, and increasing government support.

**Trends**

* **Shift towards electric mobility:** Governments and consumers are increasingly recognizing the benefits of electric vehicles, leading to a shift away from gasoline-powered vehicles.
* **Advancements in EV technology:** Innovations in battery technology, charging infrastructure, and EV design are enhancing EV performance, range, and affordability.
* **Growing popularity of subscription models:** Subscription-based services are gaining traction in the EV market, providing consumers with flexible and affordable access to electric vehicles.
* **Rise of electric commercial vehicles:** The adoption of electric vehicles is expanding into commercial sectors, including delivery, ride-hailing, and logistics.

**Key Players**

Major players in the EV market include:

* Tesla
* BYD
* Volkswagen
* Ford
* General Motors
* Toyota
* Hyundai
* Kia
* Canoo EV

**Canoo EV's Product Strategy**

Canoo EV differentiates itself in the market by focusing on modular platform architecture and innovative designs. The company's modular platform allows for the development of a variety of vehicle types, including delivery vans, lifestyle vehicles, and multipurpose vehicles. Canoo EV's designs prioritize space utilization, versatility, and sustainability.

**Canoo EV's Marketing Strategy**

Canoo EV employs a multi-channel marketing strategy to reach its target audience. The company utilizes a combination of online and offline channels, including:

* Digital marketing: Search engine optimization (SEO), social media, and paid advertising
* Experiential marketing: Pop-up events, test drives, and ride-and-drive experiences
* Partnerships with dealerships and ride-sharing companies
* Public relations: Media outreach and industry events

**Canoo EV's Financial Performance**

As a publicly traded company, Canoo EV reports its financial performance regularly. The company's revenue has been growing steadily, and it recently reported a significant increase in vehicle deliveries. Canoo EV's financial performance is closely tied to the overall growth and demand in the EV market.

**Canoo EV's Future Outlook**

Canoo EV is well-positioned to capitalize on the growing EV market. The company's focus on innovation, strategic partnerships, and subscription-based business model provides a competitive advantage. Canoo EV has a strong pipeline of upcoming vehicle models and a clear path to profitability.**Canoo's Competitors**

Canoo faces competition from a number of established electric vehicle manufacturers, as well as startups that are also developing new electric vehicles. Some of Canoo's key competitors include:

* **Tesla** is the world's leading electric vehicle manufacturer, with a lineup of vehicles that includes the Model S, Model 3, Model X, and Model Y. Tesla is also known for its advanced Autopilot driver assistance system and its extensive network of Supercharger charging stations.
* **Rivian** is a startup that is developing a lineup of electric pickup trucks and SUVs. Rivian's vehicles are designed to be rugged and off-road capable, and they come with a variety of features that are tailored to outdoor enthusiasts.
* **Lucid** is a startup that is developing a lineup of luxury electric vehicles. Lucid's vehicles are designed to be spacious, comfortable, and technologically advanced.
* **NIO** is a Chinese electric vehicle manufacturer that is known for its high-performance vehicles. NIO's vehicles come with a variety of features that are designed to appeal to luxury buyers, such as advanced infotainment systems and premium interiors.
* **Xpeng** is a Chinese electric vehicle manufacturer that is known for its affordable vehicles. Xpeng's vehicles come with a variety of features that are designed to appeal to budget-minded consumers, such as long range and fast charging capabilities.

**Market Share**

According to a report by CleanTechnica, Tesla is the world's leading electric vehicle manufacturer, with a market share of 21%. Rivian is the second-largest electric vehicle manufacturer, with a market share of 11%. Lucid is the third-largest electric vehicle manufacturer, with a market share of 10%. NIO is the fourth-largest electric vehicle manufacturer, with a market share of 8%. Xpeng is the fifth-largest electric vehicle manufacturer, with a market share of 7%.

**Products or Services Offered**

Canoo offers a lineup of electric vehicles that includes the Lifestyle Vehicle, the MPDV, and the Pickup Truck. The Lifestyle Vehicle is a compact electric SUV that is designed for urban commuting. The MPDV is a larger electric van that is designed for commercial use. The Pickup Truck is an electric pickup truck that is designed for both work and play.

**Pricing Strategies**

Canoo's pricing strategies vary depending on the model and trim level. The Lifestyle Vehicle starts at $34,750. The MPDV starts at $33,000. The Pickup Truck starts at $39,950.

**Promotion Strategies**

Canoo uses a variety of promotion strategies to reach its target market. These strategies include:

* **Online advertising:** Canoo uses online advertising to reach potential customers who are searching for information about electric vehicles.
* **Social media:** Canoo uses social media to connect with potential customers and share information about its products and services.
* **Public relations:** Canoo uses public relations to generate positive media coverage of its products and services.
* **Events:** Canoo participates in a variety of events, such as trade shows and auto shows, to showcase its products and services to potential customers.## Key Trends in the EV Market

The electric vehicle (EV) market is experiencing significant growth, driven by a variety of factors. Here are some key trends that are shaping the industry:

**Changes in Consumer Behavior**

* Increasing environmental consciousness: Consumers are becoming more aware of the environmental impact of their transportation choices and are seeking out ways to reduce their carbon footprint.
* Growing demand for affordable and fuel-efficient vehicles: As the cost of owning and operating an EV continues to decline, more consumers are considering them as a viable option.
* Technological advancement: The development of new battery technologies and the improvement of charging infrastructure has helped to address some of the concerns that consumers have about the practicality of owning an EV.

**Technological Advancements**

* Longer driving range: New battery technologies have extended the range of the average EV, making them more convenient for daily use.
* Faster charging times: The development of new charging technologies has reduced the time it takes to charge an EV, making them more convenient for on-the-go charging.
* Advanced safety features: As the EV market matures, manufacturers are incorporating more advanced safety features into their vehicles, including lane departure warning systems, automatic emergency braking, and adaptive cruise control.

**Shifts in the Competitive Landscape**

* The EV market is becoming increasingly competitive, with new players entering the market and established automakers investing heavily in developing their own EV offerings.
* Traditional automakers are partnering with technology companies to develop new and innovative EV technologies.

## Future Outlook

The EV market is expected to continue to grow in the coming years, driven by the increasing demand for sustainable and efficient transportation options. However, the market may face some challenges, such as the high cost of EV batteries and the need for a more robust charging infrastructure. Nevertheless, the long-term outlook for the EV market is positive, as governments and consumers continue to support the transition to a more sustainable transportation system.**Canoo EV's Financial Performance**

**Revenue**

* $1.2 million in revenue reported in the first quarter of 2022
* $25 million in revenue from sales and distribution channels as of 2023

**Profit Margins**

* Not currently profitable

**Return on Investment (ROI)**

* Financial data was not available to calculate ROI

**Expense Structure**

* **Product Development:** Focused on designing and developing electric vehicles with innovative features, such as a flat "skateboard" platform.
* **Manufacturing:** outsources vehicle production to VDL Nedcar in the Netherlands.
* **Marketing and Sales:** Uses a combination of online and offline channels to reach customers.
* **General and Administrative (G&A):** Includes salaries, rent, and other operating expenses.

**Canoo EV's Financial Performance Drivers**

**Product Strategy**

* Focus on developing modular electric vehicles that can be customized to meet various needs.
* Unique "skateboard" platform enables maximum interior space and adaptability.

**Business Model**

* Subscription-based model offers flexibility and convenience to customers.
* All-inclusive subscription includes maintenance, warranty, registration, and access to insurance and vehicle charging.

**Market Positioning**

* Target market includes both consumers and businesses.
* Emphasis on affordable and sustainable electric vehicles.

**Challenges**

* Competition from established automakers
* High development and production costs
* Limited charging infrastructure
* Supply chain disruptions

**Market Outlook**

* The electric vehicle market is expected to grow significantly in the coming years, driven by factors such as:
    * Government incentives
    * Consumer demand for sustainable transportation
    * Technological advancements and cost reductions

Canoo EV is well-positioned to benefit from this market growth by leveraging its unique product offerings and innovative business model. However, the company faces strong competition and will need to execute its strategy effectively to succeed.